In [ ]:
from sklearn import datasets
import pandas as pd

# Load the boston house-prices dataset (regression).
boston = datasets.load_boston()
boston_target_name = 'MEDV'
boston_features_names = boston.feature_names
boston_df = pd.DataFrame(boston.data, columns=boston_features_names)
boston_df[boston_target_name] = boston.target

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

seed = 123456
X, y = boston_df[boston_features_names], boston_df[boston_target_name]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=seed)

pipeline = Pipeline([
    ('Xgbr', XGBRegressor(objective='reg:linear',
                          colsample_bytree=0.3,
                          learning_rate=0.1,
                          max_depth=3,
                          alpha=10,
                          n_estimators=10,
                          seed=seed))
])
pipeline.fit(X_train, y_train)

In [ ]:
# Predict the first row of test data, the result should be identical to one produced by PMML
print(X_test.head(1))
pipeline.predict(X_test.head(1))

In [ ]:
from nyoka import xgboost_to_pmml

# Export the pipeline model into PMML
model = 'xgbr_pmml.pmml'
xgboost_to_pmml(pipeline, boston_features_names, boston_target_name, model)

In [ ]:
from daas_client import DaasClient

# Please, change to your URL of Daas server, and credentials
url = 'https://192.168.64.3:31753'
username = 'admin'
password = 'password'
project = 'Examples'

# Initiate a client of DaaS server, and set the created "Examples" project
client = DaasClient(url, username, password)
if not client.project_exists(project):
    client.create_project(project, 'examples', 'This is an example project')
client.set_project(project)

In [ ]:
from pprint import pprint

model_name = 'pmml-reg'

# Publish the built model into DaaS
publish_resp = client.publish(model,
                              name=model_name,
                              mining_function='regression',
                              x_test=X_test,
                              y_test=y_test,
                              description='A PMML regression model')
pprint(publish_resp)

In [ ]:
# Try to test the published model
test_resp = client.test(model_name, model_version=publish_resp['model_version'])
pprint(test_resp)

In [ ]:
# Call the test REST API above, 'model_name' is required in payload because the test runtime serves multiple models 
# in a project. 
import requests

bearer_token = 'Bearer {token}'.format(token=test_resp['access_token'])
payload = {
    'args': {'X': [{'AGE': 89.5,
                    'B': 396.9,
                    'CHAS': 0.0,
                    'CRIM': 22.5971,
                    'DIS': 1.5184,
                    'INDUS': 18.1,
                    'LSTAT': 31.99,
                    'NOX': 0.7,
                    'PTRATIO': 20.2,
                    'RAD': 24.0,
                    'RM': 5.0,
                    'TAX': 666.0,
                    'ZN': 0.0}],
             'model_name': model_name,
             'model_version': publish_resp['model_version']}}

response = requests.post(test_resp['endpoint_url'],
                         headers={'Authorization': bearer_token},
                         json=payload,
                         verify=False)
pprint(response.json())

In [ ]:
# Deploy the published model into product
deploy_resp = client.deploy(model_name, 
                            deployment_name=model_name + '-svc',
                            model_version=publish_resp['model_version'])
pprint(deploy_resp)

In [ ]:
# Call the product REST API above, the deployment runtime(s) serve the deployed model dedicatedly.
deploy_bearer_token = 'Bearer {token}'.format(token=deploy_resp['access_token'])
deploy_payload = {'args': {'X': [{'AGE': 89.5,
                                  'B': 396.9,
                                  'CHAS': 0.0,
                                  'CRIM': 22.5971,
                                  'DIS': 1.5184,
                                  'INDUS': 18.1,
                                  'LSTAT': 31.99,
                                  'NOX': 0.7,
                                  'PTRATIO': 20.2,
                                  'RAD': 24.0,
                                  'RM': 5.0,
                                  'TAX': 666.0,
                                  'ZN': 0.0}]}}

response = requests.post(deploy_resp['endpoint_url'],
                         headers={'Authorization': deploy_bearer_token},
                         json=deploy_payload,
                         verify=False)
pprint(response.json())